In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.utils import shuffle
from sklearn.metrics import log_loss 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import warnings
warnings.simplefilter(action='ignore')

In [3]:
test = pd.read_csv('test_label_enc.csv')
train = pd.read_csv('train_label_enc.csv')

In [4]:
# Aca saco las columnas que no quiero que las tome mi algoritmo.
features = list(set(train.columns) - set(['ID','Stage','Sales_Contract_No','contrato','Opportunity_ID','Opportunity_Created_Date','Account_Created_Date','Opportunity_Name',\
                                        'Price', 'Currency', 'ASP', 'ASP_converted_Currency', 'ASP_(converted)',\
                                        'Planned_Delivery_Start_Date','Size', 'Planned_Delivery_End_Date', 'Month',\
                                         'Delivery_Year', 'Actual_Delivery_Date','Product_Type','Product_Category_B',\
                                        'Total_Amount_Currency', 'Total_Amount','Last_Activity',\
                                        'Quote_Expiry_Date', 'Last_Modified_Date',\
                                        'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount','Account_Created_Date'\
                                          ,'Total_moneda_local','ASP_Currency','Last_Modified_By','Brand','Bureaucratic_Code',\
                                         'tiempo_creacion_ulti_mod','ASP_converted','Product_Name','Opportunity_Owner',\
                                          'Account_Owner','Product_Family','Account_Name']))
#features

In [5]:
train_pos= train.loc[train['Stage']==1,:].sample(7315)
train_neg = train.loc[train['Stage']==0,:].sample(7315)
train=pd.concat([train_pos,train_neg]).drop_duplicates(keep=False)
train= shuffle(train)

In [6]:
X = train[features]
Y = train['Stage']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [7]:
xgb1 = xgb.XGBClassifier()

In [8]:
xgb1.fit(x_train[features], y_train)

[16:28:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [9]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = True):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2)
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)[:,1]
    else:
        pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [10]:
#model = xgb.XGBClassifier()
#param_grid = {
#    'n_estimators': [10,50, 90],
#    'max_depth': [15,20,25],
#    'num_leaves': [50, 100, 200],
#}

#model, pred = algorithm_pipeline(x_train, x_test, y_train, y_test, model, 
#                                 param_grid, cv=5, scoring_fit='neg_log_loss')

#print(model.best_score_)
#print(model.best_params_)

In [12]:
x_test['prediccion']=xgb1.predict_proba(x_test[features])[:,1]

In [13]:
x_test['real'] = y_test

In [14]:
log_loss( x_test['real'].astype('float'),x_test['prediccion'].astype('float'))

0.2323120538568204

In [15]:
xgb1.fit(train[features], train['Stage'])

[16:34:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
pred = xgb1.predict_proba(test[features])[:,1]

In [18]:
test['target']=pred

In [19]:
test[['Opportunity_ID', 'target']]

,Opportunity_ID,target
0,10689,0.721576
1,10689,0.721576
2,10689,0.721576
3,10690,0.585649
4,10690,0.541556
...,...,...
2546,12364,0.951519
2547,12365,0.649178
2548,12366,0.593233
2549,12367,0.608683


In [20]:
test.groupby(['Opportunity_ID'], as_index=False)['target'].mean()[['Opportunity_ID', 'target']]

,Opportunity_ID,target
0,10689,0.721576
1,10690,0.568012
2,10691,0.733582
3,10692,0.847598
4,10693,0.920891
...,...,...
1562,12364,0.951519
1563,12365,0.649178
1564,12366,0.593233
1565,12367,0.608683


In [21]:
test =test.groupby(['Opportunity_ID'], as_index=False)['target'].mean()[['Opportunity_ID', 'target']]

In [22]:
test['target'].mean()

0.5040866

In [23]:
test[['Opportunity_ID', 'target']].to_csv("subidas/XGBoost_label_completo.csv", index=False)

In [424]:
#train_pos=train.loc[train['Stage']==1].sample(5000)
#train_neg=train.loc[train['Stage']==0].sample(4719)
#mi_train_completo=pd.concat([train_pos,train_neg]).drop_duplicates(keep=False)
#mi_train_completo= shuffle(mi_train_completo)
#mi_train_completo=mi_train_completo.sort_index()

In [425]:
#train=mi_train_completo

In [426]:
test.head()

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Estación,Total_moneda_local,total_en_dolares,monto_en_dolares,monto_en_dolares_max,monto_en_dolares_min,monto_en_dolares_mean,monto_en_dolares_std,contrato,precio_por_items
0,10689,3,6140,2,15,1,1,1,1,0,...,2,367419.0,4.155509e+05,183493.44,183493.44,88923.744,138516.9630,47453.583431,Con Contrato,138516.9630
3,10690,5,6118,2,15,1,1,1,1,0,...,2,757783.5,8.570531e+05,275240.16,275240.16,127196.784,171410.6277,61244.287928,Con Contrato,171410.6277
8,10691,1,11530,0,27,1,1,0,0,0,...,2,21037.5,2.103750e+04,21037.50,21037.50,21037.500,21037.5000,NaN,Con Contrato,21037.5000
9,10692,6,20314,0,28,1,1,1,0,0,...,2,2169106.5,2.169106e+06,342490.50,456654.00,228327.000,361517.7500,85939.160738,Con Contrato,361517.7500
15,10693,1,7374,0,28,0,0,0,0,0,...,2,5752.5,5.752500e+03,5752.50,5752.50,5752.500,5752.5000,NaN,Con Contrato,5752.5000


In [427]:
train.rename(columns={'Estación':'Estacion'},inplace=True)
test.rename(columns={'Estación':'Estacion'},inplace=True)

In [428]:
categorical_features = ['Region','Territory','Bureaucratic_Code','Source ','Billing_Country', 'Account_Name','Account_Owner',\
                            'Opportunity_Owner','Account_Type', 'Opportunity_Type', 'Quote_Type','Delivery_Quarter', \
                            'Delivery_Terms','Last_Modified_By',\
                            'Product_Family', 'Product_Name','ASP_Currency','Prod_Category_A', 'Hemisferio', 'Estacion']

In [429]:
# Aca saco las columnas(labels que no quiero que las tome mi algoritmo)
features = list(set(train.columns) - set(['ID','Stage','contrato','Sales_Contract_No','Opportunity_ID','Opportunity_Created_Date','Account_Created_Date','Opportunity_Name',\
                                        'Price', 'Currency', 'ASP', 'ASP_(converted)_Currency', 'ASP_(converted)',\
                                        'Planned_Delivery_Start_Date','Size', 'Planned_Delivery_End_Date', 'Month',\
                                         'Delivery_Year', 'Actual_Delivery_Date','Product_Type','Product_Category_B',\
                                        'Total_Amount_Currency', 'Total_Amount','Last_Activity',\
                                        'Quote_Expiry_Date', 'Last_Modified_Date',\
                                        'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount','Account_Created_Date','Total_moneda_local'\
                                          ,'Account_Name','ASP_Currency','Last_Modified_By']))
features

['TRF',
 'Account_Type',
 'Delivery_Quarter',
 'Pricing, Delivery_Terms_Approved',
 'Product_Family',
 'Product_Name',
 'Bureaucratic_Code_0_Approval',
 'total_en_dolares',
 'Territory',
 'Quote_Type',
 'Bureaucratic_Code_0_Approved',
 'Estacion',
 'monto_en_dolares',
 'monto_en_dolares_max',
 'Prod_Category_A',
 'Region',
 'Billing_Country',
 'monto_en_dolares_min',
 'Delivery_Terms',
 'Source ',
 'Pricing, Delivery_Terms_Quote_Appr',
 'monto_en_dolares_std',
 'Hemisferio',
 'Account_Owner',
 'Submitted_for_Approval',
 'Brand',
 'Bureaucratic_Code',
 'Cantidad de Items Asociados',
 'monto_en_dolares_mean',
 'Opportunity_Owner',
 'Opportunity_Type',
 'precio_por_items']

In [430]:
X = train[features]
Y = train['Stage']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [431]:
x_train

,TRF,Account_Type,Delivery_Quarter,"Pricing, Delivery_Terms_Approved",Product_Family,Product_Name,Bureaucratic_Code_0_Approval,total_en_dolares,Territory,Quote_Type,...,Hemisferio,Account_Owner,Submitted_for_Approval,Brand,Bureaucratic_Code,Cantidad de Items Asociados,monto_en_dolares_mean,Opportunity_Owner,Opportunity_Type,precio_por_items
15501,0,2,3,1,97,110,0,2.759570e+03,31,1,...,0,32,0,24,4,3,9.198568e+02,37,22,9.198568e+02
9048,0,0,2,1,200,3,0,1.869920e+03,31,1,...,0,15,0,24,4,1,1.869920e+03,18,1,1.869920e+03
4493,0,0,3,1,209,15,0,2.881293e+05,24,1,...,0,2,0,24,4,3,9.604311e+04,4,23,9.604311e+04
9668,0,2,0,1,35,144,1,2.216035e+05,31,1,...,0,36,0,24,4,1,2.216035e+05,18,23,2.216035e+05
2687,1,4,1,0,200,442,1,5.414500e+05,27,1,...,0,30,0,24,5,1,5.414500e+05,35,11,5.414500e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14818,0,0,2,1,212,9,1,6.821600e+03,15,1,...,0,44,0,24,4,3,2.273867e+03,8,23,2.273867e+03
3373,0,2,0,1,200,3,1,4.472000e+04,27,1,...,0,30,0,24,4,1,4.472000e+04,35,1,4.472000e+04
7764,4,2,0,1,36,136,1,1.650118e+06,22,1,...,0,4,0,24,4,1,1.650118e+06,6,1,1.650118e+06
15055,1,2,3,0,35,139,0,3.492263e+05,31,1,...,0,36,0,24,4,1,3.492263e+05,41,1,3.492263e+05


In [432]:
xgb1 = xgb.XGBClassifier()

In [462]:
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)
scores = cross_val_score(xgb1, train[features], train['Stage'], cv=10,scoring='neg_log_loss',error_score='raise' )
abs(scores)

array([0.39960446, 0.46643339, 0.43099785, 0.44942444, 0.45555164,
       0.53367681, 0.51206274, 0.42595226, 0.37564744, 0.43819274])

In [463]:
abs(scores.mean())

0.44875437725773626

In [435]:
""""estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

grid_search.fit(X, Y)"""

'"estimator = XGBClassifier(\n    objective= \'binary:logistic\',\n    nthread=4,\n    seed=42\n)\n\nparameters = {\n    \'max_depth\': range (2, 10, 1),\n    \'n_estimators\': range(60, 220, 40),\n    \'learning_rate\': [0.1, 0.01, 0.05]\n}\n\ngrid_search = GridSearchCV(\n    estimator=estimator,\n    param_grid=parameters,\n    scoring = \'roc_auc\',\n    n_jobs = 10,\n    cv = 10,\n    verbose=True\n)\n\ngrid_search.fit(X, Y)'

In [436]:
#grid_search.best_estimator_

In [437]:
#xgb1.fit(x_train[features], y_train)
xgb1.fit(X,Y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [438]:
#Veo que esta considerando como importante de mas a menos mi algoritmo
pd.DataFrame(list(zip(x_train[features], xgb1.feature_importances_))).sort_values(by=1,ascending=False).head(50)

,0,1
25,Brand,0.140845
28,monto_en_dolares_mean,0.095070
17,monto_en_dolares_min,0.089789
29,Opportunity_Owner,0.088028
26,Bureaucratic_Code,0.073944
16,Billing_Country,0.063380
5,Product_Name,0.054577
4,Product_Family,0.047535
23,Account_Owner,0.044014
1,Account_Type,0.042254


In [439]:
x_test['prediccion']=xgb1.predict_proba(x_test[features])[:,1]

In [440]:
x_test['prediccion']

283      0.745321
6777     0.161422
1779     0.635879
8998     0.772919
7447     0.893918
           ...   
6402     0.752102
4237     0.070359
11112    0.586895
13484    0.208833
5108     0.217530
Name: prediccion, Length: 1951, dtype: float32

In [441]:
x_test['real'] = y_test

In [442]:
x_test.head()

,TRF,Account_Type,Delivery_Quarter,"Pricing, Delivery_Terms_Approved",Product_Family,Product_Name,Bureaucratic_Code_0_Approval,total_en_dolares,Territory,Quote_Type,...,Submitted_for_Approval,Brand,Bureaucratic_Code,Cantidad de Items Asociados,monto_en_dolares_mean,Opportunity_Owner,Opportunity_Type,precio_por_items,prediccion,real
283,0,0,0,0,200,3,0,1.112602e+05,31,0,...,0,24,4,1,1.112602e+05,18,1,1.112602e+05,0.745321,1
6777,4,2,2,1,111,337,1,2.670051e+06,42,1,...,0,24,4,2,1.335026e+06,16,1,1.335026e+06,0.161422,0
1779,0,0,0,0,89,269,0,1.671516e+05,27,1,...,0,24,4,1,1.671516e+05,39,23,1.671516e+05,0.635879,1
8998,0,0,2,0,19,90,0,9.500400e+04,30,1,...,0,24,4,1,9.500400e+04,12,23,9.500400e+04,0.772919,1
7447,0,0,3,1,35,144,0,5.018667e+03,31,1,...,0,24,4,1,5.018667e+03,37,22,5.018667e+03,0.893918,1


In [443]:
len(y_test)

1951

In [444]:
len(x_test)

1951

In [445]:
x_test['accuracy'] = abs(x_test['real']-x_test['prediccion'])

In [446]:
x_test['accuracy'].mean()

0.2699886092690341

In [447]:
log_loss( x_test['real'].astype('float'),x_test['prediccion'].astype('float'))

0.3872834235329665

In [448]:
(sum(-( x_test['real'].astype('float')*np.log(x_test['prediccion'].astype('float')) +\
  (1- x_test['real'].astype('float'))*(np.log(1-x_test['prediccion'].astype('float'))))))/len(x_test)

0.3872834235329657

In [449]:
len(test)

1567

In [450]:
test['prediccion']=xgb1.predict_proba(test[features])[:,1]

In [451]:
test['target']=test['prediccion']

In [452]:
test['target'].value_counts()

0.055005    14
0.051990    11
0.031064    10
0.052450    10
0.007627     9
            ..
0.050114     1
0.153258     1
0.050858     1
0.003545     1
0.126953     1
Name: target, Length: 1218, dtype: int64

In [453]:
test[['Opportunity_ID', 'target']]

,Opportunity_ID,target
0,10689,0.014931
3,10690,0.010902
8,10691,0.049775
9,10692,0.011372
15,10693,0.082734
...,...,...
2545,12364,0.070492
2547,12365,0.046210
2548,12366,0.030381
2549,12367,0.009451


In [455]:
test[['Opportunity_ID', 'target']].to_csv("subidas/xgb_label_enc_v12.csv", index=False)